## RAG pipeline from Scratch

#### The Goal of RAG is to take information and pass it to an LLM so that it can generate output based on that information 

* Retrieval - Find relavent info from a query
* Augmented - Augment out input to the LLM
* Generation - Generative output from LLM

## STEPS

1. Open a PDF document (even a collection of PDFs)
2. Format the text of the PDF textbook ready for an embedding model.
3. Embed all of the chunks of text in the textbook and turn them into numerical embeddings which can store for later.
4. Build a retrieval system that uses vector search to find relevant chunck of text based ona aquery
5. Create a prompt that incorporates the relevant pieces of text.
6. Generat an answer to a query based on the passages of the textbook with an LLM

In [2]:
# downloading requirements
!pip install PyMuPDF
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 65.9 MB/s eta 0:00:00:00:0100:01


In [3]:
# Step 1
import os

# get pdf path
pdf_path = "/kaggle/input/nutrition-rag/human-nutrition-text.pdf"

In [4]:
# Step 2

import fitz
from tqdm.auto import tqdm 

def text_formatter(text: str) ->str:
    cleaned_text = text.replace("\n"," ").strip()
    
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    
    for page_number, page in tqdm(enumerate(doc)):
#         print(page_number)
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 41,
                               "page_char_count": len(text),
                               "page_word_count": len(text.split(" ")),
                               "page_sentence_count_raw": len(text.split(".")),
                               "page_token_count": len(text)/4, # 1 toke  is ~4 characters
                               "text":text})
        
    return pages_and_texts
    
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [5]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,3,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [6]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,14.180464,287.148593
std,348.86387,560.441673,95.747365,9.544587,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,8.000000,190.687500
50%,562.50000,1232.500000,215.000000,13.000000,308.125000
75%,864.25000,1605.250000,271.250000,19.000000,401.312500
max,1166.00000,2308.000000,429.000000,82.000000,577.000000


In [10]:
# Splitting text in page into sentences, using NLP lib - spaCy
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item['text']).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    #Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [12]:
import random

random.sample(pages_and_texts, k=1)

[{'page_number': 664,
  'page_char_count': 974,
  'page_word_count': 137,
  'page_sentence_count_raw': 16,
  'page_token_count': 243.5,
  'text': 'occur in Africa and Southeast Asia. The World Bank states five key  interventions to combat anemia:4  • Provide at-risk groups with iron supplements.  • Fortify staple foods with iron and other micronutrients whose  deficiencies are linked with anemia.  • Prevent the spread of malaria and treat the hundreds of  millions with the disease.  • Provide insecticide-treated bed netting to prevent parasitic  infections.  • Treat parasitic-worm infestations in high-risk populations.  Also, there is ongoing investigation as to whether supplying iron  cookware to at-risk populations is effective in preventing and  treating iron-deficiency anemia.  Learning Activities  Technology Note: The second edition of the Human  4.\xa0Anemia. The World Bank.\xa0http:/ /web.worldbank.org/ WBSITE/EXTERNAL/TOPICS/ EXTHEALTHNUTRITIONANDPOPULATION/ EXTPHAAG/ 0,,conten

In [18]:
# STEP 3

# Chucking our sentences together
# Chucking - grouping sentences into groups

# We do this because, to make our text chunks fit into our embedding model context window

num_sentence_chunk_size = 10

# eg: [25] => [10,10,5] 

def split_list(input_list:list[str],
              slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list),slice_size)]

# Testing
test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [20]:
# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list = item["sentences"],
                                        slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [21]:
random.sample(pages_and_texts, k=1)

[{'page_number': 925,
  'page_char_count': 1823,
  'page_word_count': 306,
  'page_sentence_count_raw': 26,
  'page_token_count': 455.75,
  'text': 'The Anorexia of Aging  In addition to concerns about obesity among senior citizens, being  underweight can be a major problem. A condition known as the  anorexia of aging is characterized by poor food intake, which results  in dangerous weight loss. This major health problem among the  elderly leads to a higher risk for immune deficiency, frequent falls,  muscle loss, and cognitive deficits. Reduced muscle mass and  physical activity mean that older adults need fewer calories per  day to maintain a normal weight. It is important for health care  providers to examine the causes for anorexia of aging among their  patients, which can vary from one individual to another.  Understanding why some elderly people eat less as they age can  help healthcare professionals assess the risk factors associated with  this condition. Decreased intake may be